# Boundary value problem example: A ball thrown in the air

Consider a ball thrown in the air. The height of the ball $x$ is given by (neglecting air friction): 
$$\frac{d^2x}{dt^2}=-g.$$
As default initial condition, we will take is $x(0)=0$, and we will try to find the initial velocity such that $x(10) = 0$. We first do the normal trick to turn the second-order ODE into two coupled first-order ODEs:
$$\frac{dx}{dt}=v, \; \; \; \frac{dv}{dt}=-g.$$

This example was taken from Newman Sec. 8.6.

In [1]:
import numpy as np


class boundaryValue:
    '''Class for solving our boundary value problem of a thrown ball.'''

    def __init__(self,t0=0.0,x0=0.0,v_guess=[1.0,5.0],tf=10.0,nRKsteps=1000,accuracy=1e-10):
        self.g=9.81 # Acceleration of gravity in m/s^2
        self.x0=x0
        self.accuracy=accuracy
        self.rkPoints = np.linspace(t0,tf,nRKsteps)
        self.deltaT=(tf-t0)/nRKsteps
        
        self.v0=self.secant_root(self.rk4_height,v_guess[0],v_guess[1],accuracy)
        
    def secant_root(self,func,v0,v1,accuracy):
        '''Calculate the root with the Secant method'''
        maxSteps=1000 # Limit the number of steps in case of divergence
        
        for step in range(maxSteps):
        
            if self.first_derivative(func,v0,v1) < 1.0e-20:
                raise('Derivative is zero, try different initial guesses.')
                
            else:
                v2=v1-func(v1)/self.first_derivative(func,v0,v1)

            if abs(v2) < 1.0e-16 and  abs(v2-v1) < accuracy:
                return v2


            elif abs(v2-v1) < abs(v2*accuracy):
                return v2
            
            v0=v1
            v1=v2
            
        # If we end up here, out root did not converge
        raise('Root did not converge')
        
    def first_derivative(self,func,v0,v1):
        '''Caluclate the numerical first derivative of the function''' 
        return (func(v1)-func(v0))/(v1-v0)
        
    def rk4_height(self,v):
        '''Evaluating the height for a given velocity using the RK method'''
        r=np.array([self.x0,v])
        dt=self.deltaT
        
        for time in self.rkPoints:
            k1=dt*self.f(r)
            k2=dt*self.f(r+0.5*k1)
            k3=dt*self.f(r+0.5*k2)
            k4=dt*self.f(r+k3)
            r+=(k1+2.0*k2+2.0*k3+k4)/6.0
        
        return r[0]    
        
    def f(self,r):
        '''RHS for Runge-Kutta method, two variables'''
        x=r[0]
        v=r[1]
        fx=v
        fy=-self.g
        return np.array([fx,fy])
           

In [4]:
ballThrow=boundaryValue()
print(ballThrow.v0)

print(ballThrow.rk4_height(ballThrow.v0))

49.050000000000225
-1.5487611193520934e-14
